In [1]:
import nibabel as nb
import numpy as np
from skimage.measure import label
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getLargestCC(segmentation):
    labels = label(segmentation)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC

def create_if_not(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
df = pd.read_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/0_0.0_concat_report_final.csv')

with open('/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/UKB/test_volumes.txt', mode='r') as file_handler:
    volumes = file_handler.read().splitlines()

projects = ['MC_dropout_quicknat', 'probabilistic_quicknat']  #'hierarchical_quicknat', 'full_bayesian'

dicts = {}
for p in projects:
    dicts[p] = {}
    for ijk, v in enumerate(volumes):
        print("\rProcessing {} {}/{}.".format(p, ijk, len(volumes)), end="")
        try: 
            if p == 'MC_dropout_quicknat':
                pfix = df[df['volume_id']==v]['target_scan_file'].values[0].split('.')[0][-1]
            else:
                pfix = ''

            data_nifty = nb.load(f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/{p}_UKB_v2/{p}_UKB_v2_predictions_UKB/{v}{pfix}_seg.nii.gz')

            header = data_nifty.header
            data = data_nifty.get_fdata()

            liver_seg =np.zeros_like(data)
            spleen_seg = np.zeros_like(data)

            cc = getLargestCC(data)
            liver_seg[cc] = 2

            data[cc] = 0
            if np.unique(data)[1] != 1:
                print('No spleen here!!! ')
                continue

            ss = getLargestCC(data)
            spleen_seg[ss] = 1

            gt = (liver_seg + spleen_seg).astype('float32')

            nifti_img = nb.MGHImage(np.squeeze(gt), np.eye(4))
            path = f'/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/{p}_UKB_v2/{p}_UKB_v2_predictions_UKB_post_processed'
            create_if_not(path)
            nb.save(nifti_img, os.path.join(path, f'{v}_seg.nii.gz'))
        except Exception as e:
            print(e)
            continue

Processing MC_dropout_quicknat 18/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 25/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 31/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 51/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 68/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 79/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 118/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 175/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 188/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 210/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 233/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quick

Processing MC_dropout_quicknat 1177/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1181/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1203/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1216/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1233/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1273/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1287/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1290/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1306/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1315/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 1317/19853.index 0 is out of bounds for axis 0 with size 0
Processing

Processing MC_dropout_quicknat 2458/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2461/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2466/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2467/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2473/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2488/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2498/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2504/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2517/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2523/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 2525/19853.index 0 is out of bounds for axis 0 with size 0
Processing

Processing MC_dropout_quicknat 3529/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3543/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3549/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3556/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3561/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3568/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3598/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3603/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3631/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3693/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 3722/19853.index 0 is out of bounds for axis 0 with size 0
Processing

Processing MC_dropout_quicknat 4730/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4731/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4752/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4787/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4793/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4795/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4815/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4817/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4823/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4874/19853.index 0 is out of bounds for axis 0 with size 0
Processing MC_dropout_quicknat 4884/19853.index 0 is out of bounds for axis 0 with size 0
Processing

...

Processing probabilistic_quicknat 11067/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/3801887_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 11095/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/3810628_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 11284/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/3860929_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 11286/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/3861210_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 11377/19853.No such fi

Processing probabilistic_quicknat 13015/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4298656_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 13067/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4308720_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 13075/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4310353_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 13103/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4317940_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 13151/19853.No such fi

Processing probabilistic_quicknat 14933/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4785361_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 14962/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4794972_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 14995/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4806702_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 15024/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/4812764_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 15127/19853.No such fi

Processing probabilistic_quicknat 17090/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5333570_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 17101/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5337097_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 17143/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5344626_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 17309/19853.No spleen here!!! 
Processing probabilistic_quicknat 17369/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5399246_20201_2_0_seg

Processing probabilistic_quicknat 18965/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5805635_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 18969/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5806930_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 18990/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5814756_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 19205/19853.No such file or no access: '/home/abhijit/nas_drive/Abhijit/Jyotirmay/ukb_outs/outs/probabilistic_quicknat_UKB_v2/probabilistic_quicknat_UKB_v2_predictions_UKB/5864810_20201_2_0_seg.nii.gz'
Processing probabilistic_quicknat 19264/19853.No such fi